<a href="https://colab.research.google.com/github/ImranRiazChohan/LSTM-/blob/master/TimeSeries_Prediction_(LSTM)-.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tqdm import tqdm_notebook
from tensorflow.keras.optimizers import SGD,RMSprop,Adam
from tensorflow import keras
from tensorflow.keras.layers import Dense,LSTM,Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras import optimizers
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from tensorflow.keras.utils import normalize

In [0]:
from google.colab import files
uploaded=files.upload()

In [0]:
data_frame=pd.read_excel('2 year dataset.xlsx',date_parser=True)

In [0]:
data_frame.head(5)

In [0]:
data_frame.tail(5)

In [0]:
data_frame.set_index('Date',inplace=True)

In [0]:
data_frame.index=pd.to_datetime(data_frame.index)

In [0]:
data_frame.head()

In [0]:
data_frame.tail()

In [0]:
data_frame.count()

In [0]:
data_frame.isnull().sum()

In [0]:
data_frame['rice'].fillna(1,inplace=True)

In [0]:
# train_size=int(len(data_frame)* 0.75)
# test_size=len(data_frame)-train_size
# train_data,test_data=data_frame.iloc[0:train_size].values,data_frame.iloc[train_size:len(data_frame)].values
# print(train_data.shape,test_data.shape)

In [0]:
scaler = MinMaxScaler()
traindata= scaler.fit_transform(data_frame)
type(traindata)

In [0]:
np.shape(traindata)

In [0]:
train_size=int(len(traindata)* 0.75)
test_size=len(traindata)-train_size
train_data,test_data=traindata[0:train_size],traindata[train_size:len(traindata)]
print(train_data.shape,test_data.shape)

In [0]:
train_gent=TimeseriesGenerator(train_data,train_data,length=3,batch_size=1)

In [0]:
train_gent[0][0].shape

In [0]:
val_gent=TimeseriesGenerator(test_data,test_data,length=3,batch_size=1)

In [0]:
val_gent[0][0].shape

In [0]:
def model_LSTM():
    regressior = Sequential()
    regressior.add(LSTM(units = 120, activation = 'relu', return_sequences = True, input_shape = (3, 30)))
    regressior.add(Dropout(0.2))
    regressior.add(LSTM(units = 80, activation = 'relu',return_sequences=True))
    regressior.add(Dropout(0.2))
    
    regressior.add(LSTM(units = 80, activation = 'relu',return_sequences=True))
    regressior.add(Dropout(0.2))
    
    regressior.add(LSTM(units = 100, activation = 'relu'))
    regressior.add(Dropout(0.2))
    regressior.add(Dense(units = 30))
    regressior.compile(loss='mse',optimizer=optimizers.Adam(lr=1e-4),metrics=['mae'])
    return regressior

In [0]:
model=model_LSTM()

In [0]:
model.summary()

In [0]:
model.fit_generator(train_gent,epochs=5,verbose=1,validation_data=(val_gent))

In [0]:
pred=model.predict_generator(val_gent)
print(pred)

In [0]:
scaler.inverse_transform(pred)